In [1]:
%load_ext autoreload

%autoreload 2



In [2]:
import pandas as pd
import os
# Import libraries necessary for this analysis
from itertools import  count

#import renders as rs
from IPython.display import display # Allows the use of display() for DataFrames

from library import *
#from Models import *

import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix

import seaborn as sns

from __future__ import division

import matplotlib.pyplot as plt
from numpy import linspace, loadtxt, ones, convolve
import numpy as np
import pandas as pd
import collections
from random import randint
from matplotlib import style
style.use('fivethirtyeight')

%matplotlib inline
%matplotlib notebook
izip = zip
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [7]:
host = 'prod-pentaho.cxfaihg8elfv.eu-west-1.rds.amazonaws.com'
db = 'billin_prod'
user = 'billin'
password = 'ThisIsTheRiverOfTheNight'

db = DB(host=host, db=db, user=user, password=password)

df = db.gettable('users')

columns = df.describe().columns.tolist()

In [8]:
columns

['activated',
 'avatar',
 'created_at',
 'deleted_at',
 'email',
 'email_invoice_received',
 'email_business_role_change',
 'email_business_user_added',
 'email_chat_new_messages',
 'email_contact_new_request',
 'email_invoice_due_dates',
 'email_invoice_status_changed',
 'id',
 'lastname',
 'last_seen',
 'locked',
 'name',
 'password',
 'referrer',
 'signup_from',
 'updated_at',
 'vip',
 'expo_token',
 'sourced',
 'signup_by',
 'services_communication_permission',
 'third_party_sharing_permission']

In [84]:
temp = df.drop(labels=['password', 'email', 'created_at', 'updated_at', 'locked', 'vip', 'third_party_sharing_permission', 'name', 'lastname', 'id'], axis=1)

In [85]:
temp.describe()

,activated,avatar,deleted_at,email_invoice_received,email_business_role_change,email_business_user_added,email_chat_new_messages,email_contact_new_request,email_invoice_due_dates,email_invoice_status_changed,last_seen,referrer,signup_from,expo_token,sourced,signup_by,services_communication_permission
count,26977,24155,593,26977,26977,26977,26977,26977,26977,26977,26463,3898,26972,5,26977,1436,26977
unique,2,2415,418,2,2,2,2,2,2,2,25773,1301,7,5,2,2,2
top,True,default.png,2018-04-24 06:32:35.536777+00:00,True,True,True,True,True,True,True,2018-02-09 23:24:39.319000+00:00,6ed0d667-a98a-4618-987b-d26ccbeca9be,singup,ExponentPushToken[qkwUxJLTChUC5sx2D-U3Wo],web,93c97ded-a259-4eee-b2a7-24027842a6d8,False
freq,16736,21741,176,26799,26754,26755,26782,26780,26787,26735,9,897,17263,1,26426,1397,26967


In [86]:
temp['last_seen'] = temp.last_seen.map(lambda c: int(c.timestamp()) if c is not pd.NaT else 0 )
temp['deleted_at'] = temp.deleted_at.map(lambda c: int(c.timestamp()) if c is not pd.NaT else 0 )


In [95]:
yesterday = int(pd.datetime.today().timestamp()) - 86400
lastweek = int(pd.datetime.today().timestamp()) - (86400 * 7)
lastmonth = int(pd.datetime.today().timestamp()) - (86400 * 30)
threemonths = int(pd.datetime.today().timestamp()) - (86400 * 90)
sixmonths = int(pd.datetime.today().timestamp()) - (86400 * 180)

In [108]:
temp['last_seen_yesterday'] = temp.last_seen.map(lambda c: 1 if (c - yesterday) >= 0 else 0)
temp['last_seen_lastweek'] = temp.last_seen.map(lambda c: 1 if (c - lastweek) >= 0 else 0)
temp['last_seen_lastmonth'] = temp.last_seen.map(lambda c: 1 if (c - lastmonth) >= 0 else 0)
temp['last_seen_threemonths'] = temp.last_seen.map(lambda c: 1 if (c - threemonths) >= 0 else 0)
temp['last_seen_sixmonths'] = temp.last_seen.map(lambda c: 1 if (c - sixmonths) >= 0 else 0)

temp['deleted_at_yesterday'] = temp.deleted_at.map(lambda c: 1 if (c - yesterday) >= 0 else 0)
temp['deleted_at_lastweek'] = temp.deleted_at.map(lambda c: 1 if (c - lastweek) >= 0 else 0)
temp['deleted_at_lastmonth'] = temp.deleted_at.map(lambda c: 1 if (c - lastmonth) >= 0 else 0)
temp['deleted_at_threemonths'] = temp.deleted_at.map(lambda c: 1 if (c - threemonths) >= 0 else 0)
temp['deleted_at_sixmonths'] = temp.deleted_at.map(lambda c: 1 if (c - sixmonths) >= 0 else 0)
temp.drop(columns=['last_seen', 'deleted_at'], inplace=True)

In [112]:
pd.get_dummies(temp.activated).head()

,False,True
0,1,0
1,0,1
2,0,1
3,0,1
4,0,1


In [142]:
temp.activated = temp.activated.astype(int)
temp.email_invoice_received = temp.email_invoice_received.astype(int)
temp.email_business_role_change = temp.email_business_role_change.astype(int)
temp.email_business_user_added = temp.email_business_user_added.astype(int)
temp.email_chat_new_messages = temp.email_chat_new_messages.astype(int)
temp.email_contact_new_request = temp.email_contact_new_request.astype(int)
temp.email_invoice_due_dates = temp.email_invoice_due_dates.astype(int)
temp.email_invoice_status_changed = temp.email_invoice_status_changed.astype(int)

temp.services_communication_permission = temp.services_communication_permission.astype(int)

In [ ]:
#temp.sourced = temp.sourced.astype(int)
#temp.signup_by = temp.signup_by.astype(int)

In [125]:
temp.avatar = temp.avatar.map(lambda a: 0 if a == 'default.png' else a)
temp.avatar = temp.avatar.map(lambda a: 1 if a is None else a)
temp.avatar = temp.avatar.map(lambda a: 2 if a != 0 and a != 1 else a)

In [135]:
temp = pd.concat([temp, pd.get_dummies(temp.signup_from, prefix='signup_from_')], axis=1)
temp.drop(columns='signup_from', inplace=True)

In [144]:

temp = pd.concat([temp, pd.get_dummies(temp.sourced, prefix='sourced_')], axis=1)
temp.drop(columns='sourced', inplace=True)

In [150]:
signup_by = pd.get_dummies(temp.signup_by, dummy_na=True)
signup_by.columns = ['signup_by_A', 'signup_by_B', 'signup_by_NaN']

In [153]:
temp = pd.concat([temp, signup_by], axis=1)

In [ ]:
temp = pd.concat([temp, pd.get_dummies(temp.signup_by, prefix='signup_by_',columns=[])], axis=1)
temp.drop(columns='sourced', inplace=True)

In [158]:
temp.referrer = temp.referrer.map(lambda r: 0 if r is None else 1)

In [160]:
temp.referrer.describe()

count    26977.000000
mean         0.144493
std          0.351596
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: referrer, dtype: float64

In [136]:
temp.drop(columns=['activated','last_seen_1day','last_seen_yesterday','last_seen_lastweek','last_seen_lastmonth','last_seen_threemonths','last_seen_sixmonths','deleted_at_yesterday','deleted_at_lastweek','deleted_at_lastmonth','deleted_at_threemonths','deleted_at_sixmonths']).describe()



,avatar,signup_from__add_user,signup_from__app,signup_from__billinV2,signup_from__contact_request,signup_from__direct_signup,signup_from__invoice,signup_from__singup
count,26977.000000,26977.000000,26977.000000,26977.000000,26977.000000,26977.000000,26977.000000,26977.000000
mean,0.283575,0.023946,0.021574,0.064796,0.037254,0.126812,0.085517,0.639915
std,0.618176,0.152885,0.145290,0.246170,0.189387,0.332768,0.279655,0.480034
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [161]:
temp.head()

,activated,avatar,email_invoice_received,email_business_role_change,email_business_user_added,email_chat_new_messages,email_contact_new_request,email_invoice_due_dates,email_invoice_status_changed,referrer,...,signup_from__billinV2,signup_from__contact_request,signup_from__direct_signup,signup_from__invoice,signup_from__singup,sourced__app,sourced__web,signup_by_A,signup_by_B,signup_by_NaN
0,0,0,1,1,1,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,0,1,0,0,1
2,1,0,1,1,1,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,1
3,1,2,1,1,1,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,1
4,1,0,1,1,1,1,1,1,1,0,...,0,0,0,0,1,0,1,0,0,1


In [61]:
temp.last_seen.map(lambda c: int(c.timestamp()) if c is not pd.NaT else 0 )

0        1508500863
1        1508502242
2        1492786187
3        1497184937
4        1491415028
5                 0
6                 0
7        1491319582
8                 0
9                 0
10       1510845825
11                0
12                0
13                0
14                0
15                0
16                0
17       1491396645
18       1491398434
19       1491404352
20       1492015424
21                0
22                0
23       1505684630
24       1492591729
25       1491467044
26       1491467177
27       1491382007
28       1491480566
29       1491477927
            ...    
26947    1534710910
26948    1534742544
26949    1534692823
26950    1534693183
26951    1534694872
26952    1534711312
26953    1534747548
26954    1534715454
26955    1534728049
26956    1534745491
26957    1534716522
26958    1534747549
26959    1534729553
26960    1534733652
26961    1534751658
26962    1534750948
26963    1534750124
26964    1534745323
26965    1534745950


In [46]:
date = temp.last_seen.iloc[0]

In [57]:
int(date.timestamp())

1508500863

In [30]:
temp.referrer.unique()

array([None, '1581cf1b-1e5e-494c-a4f0-e4f564416198',
       '2d8ab5f0-65d4-4c54-90c4-a0f6f8c076c8', ...,
       '492cd077-9fa7-49e9-b779-359f896a2399',
       '969b0588-5538-44fb-8d20-b05fc2026f7b',
       '796ca27f-993e-44b7-8550-8e6cf80cbb93'], dtype=object)

In [33]:
temp[temp['id'] == '796ca27f-993e-44b7-8550-8e6cf80cbb93']


,activated,avatar,deleted_at,email_invoice_received,email_business_role_change,email_business_user_added,email_chat_new_messages,email_contact_new_request,email_invoice_due_dates,email_invoice_status_changed,id,lastname,last_seen,name,referrer,signup_from,expo_token,sourced,signup_by,services_communication_permission


In [ ]:
def ScatterMatrix(df):

    _, ax = plt.subplots(figsize=(20, 20))

    _ = pd.scatter_matrix(df.dropna(axis=0,how='any'), alpha=0.4, diagonal='kde', ax=ax)

In [ ]:
def rulePCA(X,  n=2):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=n)
    try:
        datapca = pca.fit_transform(X)
    except:
        print(X)
        raise

    return datapca, pca

dfpca = df[columns].copy()
dfpca.columns = ['d1','d2','d3']

In [ ]:
X,pca  = rulePCA(df[columns])
dforig = df[columns].copy()
dfpca = pd.DataFrame(X,columns=['d1','d2'])

dfpca['d2'] = np.nan_to_num(dfpca['d1']**2 + np.log(dfpca['d2']))
dfpca['d1'] = np.nan_to_num(np.sin(dfpca['d1']))
dfpca['d3'] = np.cos(np.nan_to_num(dfpca['d1']  - dfpca['d3']))


In [ ]:

data_as_frame = dfpca.copy()
ScatterMatrix(dfpca)


In [ ]:
def moving_average(data, window_size):
    """ Computes moving average using discrete linear convolution of two one dimensional sequences.
    Args:
    -----
            data (pandas.Series): independent variable
            window_size (int): rolling window size

    Returns:
    --------
            ndarray of linear convolution

    References:
    ------------
    [1] Wikipedia, "Convolution", http://en.wikipedia.org/wiki/Convolution.
    [2] API Reference: https://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html

    """
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(data, window, 'same')


def explain_anomalies(y, window_size, sigma=1.0):
    """ Helps in exploring the anamolies using stationary standard deviation
    Args:
    -----
        y (pandas.Series): independent variable
        window_size (int): rolling window size
        sigma (int): value for standard deviation

    Returns:
    --------
        a dict (dict of 'standard_deviation': int, 'anomalies_dict': (index: value))
        containing information about the points indentified as anomalies

    """
    avg = moving_average(y, window_size).tolist()
    residual = y - avg
    # Calculate the variation in the distribution of the residual
    std = np.std(residual)
    return {'standard_deviation': round(std, 3),
            'anomalies_dict': collections.OrderedDict([(index, y_i) for
                                                       index, y_i, avg_i in izip(count(), y, avg)
              if (y_i > avg_i + (sigma*std)) | (y_i < avg_i - (sigma*std))])}


def explain_anomalies_rolling_std(y, window_size, sigma=1.0):
    """ Helps in exploring the anamolies using rolling standard deviation
    Args:
    -----
        y (pandas.Series): independent variable
        window_size (int): rolling window size
        sigma (int): value for standard deviation

    Returns:
    --------
        a dict (dict of 'standard_deviation': int, 'anomalies_dict': (index: value))
        containing information about the points indentified as anomalies
    """
    avg = moving_average(y, window_size)
    avg_list = avg.tolist()
    residual = y - avg
    # Calculate the variation in the distribution of the residual
    testing_std = pd.rolling_std(residual, window_size)
    testing_std_as_df = pd.DataFrame(testing_std)
    rolling_std = testing_std_as_df.replace(np.nan,
                                  testing_std_as_df.ix[window_size - 1]).round(3).iloc[:,0].tolist()
    std = np.std(residual)
    return {'stationary standard_deviation': round(std, 3),
            'anomalies_dict': collections.OrderedDict([(index, y_i)
                                                       for index, y_i, avg_i, rs_i in izip(count(),
                                                                                           y, avg_list, rolling_std)
              if (y_i > avg_i + (sigma * rs_i)) | (y_i < avg_i - (sigma * rs_i))])}


# This function is repsonsible for displaying how the function performs on the given dataset.
def plot_results(x, y, window_size, sigma_value=1,
                 text_xlabel="X Axis", text_ylabel="Y Axis", applying_rolling_std=False):
    """ Helps in generating the plot and flagging the anamolies.
        Supports both moving and stationary standard deviation. Use the 'applying_rolling_std' to switch
        between the two.
    Args:
    -----
        x (pandas.Series): dependent variable
        y (pandas.Series): independent variable
        window_size (int): rolling window size
        sigma_value (int): value for standard deviation
        text_xlabel (str): label for annotating the X Axis
        text_ylabel (str): label for annotatin the Y Axis
        applying_rolling_std (boolean): True/False for using rolling vs stationary standard deviation
    """
    plt.figure(figsize=(15, 8))
    plt.plot(x, y, "k.")
    y_av = moving_average(y, window_size)
    plt.plot(x, y_av, color='green')
    #plt.xlim(0, 1000)
    plt.xlabel(text_xlabel)
    plt.ylabel(text_ylabel)

    # Query for the anomalies and plot the same
    events = {}
    if applying_rolling_std:
        events = explain_anomalies_rolling_std(y, window_size=window_size, sigma=sigma_value)
    else:
        events = explain_anomalies(y, window_size=window_size, sigma=sigma_value)
        


    x_anomaly = np.fromiter(events['anomalies_dict'].keys(), dtype=int, count=len(events['anomalies_dict']))
    y_anomaly = np.fromiter(events['anomalies_dict'].values(), dtype=float, count=len(events['anomalies_dict']))
    #plt.plot(x_anomaly, y_anomaly, "r*", markersize=12)

    # add grid and lines and enable the plot
    plt.grid(True)
    plt.show()

In [ ]:
rng = np.random.RandomState(42)

# Generate train data
X_train = data_as_frame[['d1', 'd2']].as_matrix()
X_test = data_as_frame.sample(n=1000).as_matrix()
X_outliers = data_as_frame.sample(n=100).as_matrix()

# fit the model
clf = IsolationForest(n_estimators=500, max_samples=50000, random_state=rng, n_jobs=48, contamination=0.1)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)


# plot the line, the samples, and the nearest vectors to the plane
xx, yy = np.meshgrid(np.linspace(data_as_frame['d1'].min().astype(int), data_as_frame['d1'].max().astype(int), 1000), np.linspace(data_as_frame['d2'].min().astype(int), data_as_frame['d2'].max().astype(int), 1000))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

df = pd.DataFrame(X_train)
df['output'] = y_pred_train

inliers = df.loc[df['output'] == 1].drop(columns='output').as_matrix()
outliers = df.loc[df['output'] == -1].drop(columns='output').as_matrix()

In [ ]:
_ = plt.figure(figsize=(15, 15))
plt.title("IsolationForest")

plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white', s=20, edgecolor='k', alpha=0.3)
b2 = plt.scatter(inliers[:, 0], inliers[:, 1], c='green', s=20, edgecolor='k', alpha=0.3)
c = plt.scatter(outliers[:, 0], outliers[:, 1], c='red',  s=20, edgecolor='k', alpha=0.8)
plt.axis('tight')
plt.xlim((0, data_as_frame['d1'].max()))
plt.ylim((0, data_as_frame['d2'].max()))
plt.legend([b1, b2, c],
           ["training observations",
            "new regular observations", "new abnormal observations"],
           loc="upper left")

plt.show()

In [ ]:
rng = np.random.RandomState(42)

# Generate train data
X_train = data_as_frame.as_matrix()
X_test = data_as_frame.sample(n=1000).as_matrix()
X_outliers = data_as_frame.sample(n=100).as_matrix()

# fit the model
clf = IsolationForest(n_estimators=500, max_samples=50000, random_state=rng, n_jobs=48, contamination=0.1)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)



df = pd.DataFrame(X_train)

df['output'] = y_pred_train
df.columns = ['d1', 'd2', 'output']

inliers = df.loc[df['output'] == 1].drop(columns='output').as_matrix()
outliers = df.loc[df['output'] == -1].drop(columns='output').as_matrix()

In [ ]:
dforig['output'] = df.output
dforig.columns = ['d1','d2','d3','output']

In [ ]:



Print_Report(df)


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(15, 15))


ax = fig.add_subplot(111, projection='3d')

ax.scatter(inliers[:, 0], inliers[:, 1], inliers[:, 2], c='green', edgecolor='k', alpha=0.5)
ax.scatter(outliers[:, 0], outliers[:, 1], outliers[:, 2], c='red',  s=20, edgecolor='k', alpha=0.5)



ax.set_xlabel('d1')
ax.set_ylabel('d2')
ax.set_zlabel('d3')

plt.xlim((data_as_frame['d1'].min(), data_as_frame['d1'].max()))
plt.ylim((data_as_frame['d2'].min(), data_as_frame['d2'].max()))
ax.set_zlim(data_as_frame['d3'].min(), data_as_frame['d3'].max())
plt.show()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
x, y, z = inliers[:, 0], inliers[:, 1], inliers[:, 2]

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name='Inliers',
    mode='markers',
    marker=dict(
        color='green',
        size=1.5,
        symbol='circle',
        line=dict(
            color='green',
            width=0.1
        ),
        opacity=0.7
    )
)

x2, y2, z2 = outliers[:, 0], outliers[:, 1], outliers[:, 2]
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    name='Outliers',
    mode='markers',
    marker=dict(
        color='red',
        size=2,
        symbol='circle',
        line=dict(
            color='red',
            width=0.1
        ),
        opacity=0.9
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title='Anomaly Detection for d1',
    scene = dict(
    xaxis=dict(
        title='d1',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='d2',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    zaxis=dict(
        title='d3',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [ ]:
file = './Data/Report-20180704134629.xlsx'
df = pd.read_excel(file)

In [ ]:
Cluster0 = df.loc[df['Cluster'] == 0].drop(columns='Cluster').as_matrix()
Cluster1 = df.loc[df['Cluster'] == 1].drop(columns='Cluster').as_matrix()
Cluster2 = df.loc[df['Cluster'] == 2].drop(columns='Cluster').as_matrix()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
x, y, z = Cluster0[:, 0], Cluster0[:, 1], Cluster0[:, 2]

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    name='Cluster0',
    mode='markers',
    marker=dict(
        color='green',
        size=1.5,
        symbol='circle',
        line=dict(
            color='green',
            width=0.1
        ),
        opacity=0.7
    )
)

x2, y2, z2 = Cluster1[:, 0], Cluster1[:, 1], Cluster1[:, 2]
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    name='Cluster1',
    mode='markers',
    marker=dict(
        color='red',
        size=2,
        symbol='circle',
        line=dict(
            color='red',
            width=0.1
        ),
        opacity=0.9
    )
)
x3, y3, z3 = Cluster2[:, 0], Cluster2[:, 1], Cluster2[:, 2]
trace3 = go.Scatter3d(
    x=x3,
    y=y3,
    z=z3,
    name='Cluster2',
    mode='markers',
    marker=dict(
        color='blue',
        size=2,
        symbol='circle',
        line=dict(
            color='blue',
            width=0.1
        ),
        opacity=0.9
    )
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    title='\nClustering for Securitas',
    scene = dict(
    xaxis=dict(
        title='d1',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='d2',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    zaxis=dict(
        title='d3',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
    
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')